# Индекс аффинитивности по соц-дем
Пример расчета аффинитивности целевых аудиторий

## Описание задачи и условий расчета
- Период: 01.01.2022 - 31.01.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+ в разбивке на целевые аудитории
- Место просмотра: Все места (дом+дача)
- Каналы: ТНТ и СТС									
- Статистики: TgAffin%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

Получим id нужных нам компаний

In [2]:
# выберем телесети по имени (части имени)
cats.get_tv_net(name=["ТНТ", "СТС"])

# далее в запросах будем использовать id телесетей.

Запрошены записи: 0 - 0
Всего найдено записей: 0



""


In [3]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['пол','возраст','образование','род занятий','уровень дохода','питание'])

Запрошены записи: 0 - 155
Всего найдено записей: 155



,id,valueId,name,colName,valueName
0,1,1,Пол,Sex,Мужчины
1,1,2,Пол,Sex,Женщины
2,3,1,Образование,Education,начальное
3,3,2,Образование,Education,среднее
4,3,3,Образование,Education,высшее
...,...,...,...,...,...
150,2,10,Возраст,Age,10
151,2,89,Возраст,Age,89
152,2,8,Возраст,Age,8
153,18,0,Возраст детей: 2-3 года,KidsAge4,Нет


## Формирование задания
Зададим условия расчета

In [4]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-01-01', '2022-01-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId IN (83, 11)'

# Указываем список статистик для расчета
statistics = ['TgAffinPer']

# Указываем срезы
slices = ['tvNetName']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Формируем группу значений параметров для различных ЦА

In [5]:
# Задаем необходимые группы
targets = {
    'Тотал':{'prj_name': '4+', 'targetdemo': 'age >= 4'},
    'Мужчины':{'prj_name': 'Пол', 'targetdemo': 'sex = 1'},
    'Женщины':{'prj_name': 'Пол', 'targetdemo': 'sex = 2'},
    '4-17':{'prj_name': 'Возраст', 'targetdemo': 'age >= 4 AND age <= 17'},
    '18-34':{'prj_name': 'Возраст', 'targetdemo': 'age >= 18 AND age <= 34'},
    '35-44':{'prj_name': 'Возраст', 'targetdemo': 'age >= 35 AND age <= 44'},
    '45-54':{'prj_name': 'Возраст', 'targetdemo': 'age >= 45 AND age <= 54'},
    '55+':{'prj_name': 'Возраст', 'targetdemo': 'age >= 55'},
    'Начальное, среднее':{'prj_name': 'Образование', 'targetdemo': 'education IN (1, 2)'},
    'Высшее':{'prj_name': 'Образование', 'targetdemo': 'education = 3'},
    'Руководитель':{'prj_name': 'Род занятий', 'targetdemo': 'status IN (1,9,10)'},
    'Рабочий':{'prj_name': 'Род занятий', 'targetdemo': 'status = 4'},
    'Дошкольник, студент, учащийся':{'prj_name': 'Род занятий', 'targetdemo': 'status = 5'},
    'Домохозяйка, молодая мать':{'prj_name': 'Род занятий', 'targetdemo': 'status = 8'},
    'Специалист, служащий':{'prj_name': 'Род занятий', 'targetdemo': 'status IN (2, 3)'},
    'Другое':{'prj_name': 'Род занятий', 'targetdemo': 'status IN (6,7,11)'},
    'Имеют возможность покупать только самое необходимое':{'prj_name': 'Уровень дохода', 'targetdemo': 'incLevel IN (1,2)'},
    'Хватает на еду и одежду, но не на дорогие вещи':{'prj_name': 'Уровень дохода', 'targetdemo': 'incLevel = 3'},
    'Имеют возможность покупать дорогие вещи':{'prj_name': 'Уровень дохода', 'targetdemo': 'incLevel IN (4,5)'},
    'Меньше четверти':{'prj_name': 'Затраты на питание', 'targetdemo': 'spendingsOnFood = 1'},
    'От четверти до половины':{'prj_name': 'Затраты на питание', 'targetdemo': 'spendingsOnFood = 2'},
    'От половины до трех четвертей':{'prj_name': 'Затраты на питание', 'targetdemo': 'spendingsOnFood = 3'},
    'Более трех четвертей':{'prj_name': 'Затраты на питание', 'targetdemo': 'spendingsOnFood = 4'},
    'Нет ответа':{'prj_name': 'Затраты на питание', 'targetdemo': 'spendingsOnFood = 5'}
}

## Расчет задания

In [6]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задания и отправляем на расчет
for target, logical in targets.items():
    
    # Формируем задание на расчет ЦА
    project_name = f"{logical['prj_name']}&{target}" 
    targetdemo_filter = f"{logical['targetdemo']}"
      
    # Формируем из указанных параметров задание для API TV Index в формате JSON для каждого из этапов цикла
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

Отправляем задания на расчет
........................
Ждем выполнения
Расчет задач (24) [ = = = ] время расчета: 0:00:18.235958
Расчет завершен, получаем результат
Собираем таблицу
........................

Настраиваем внешний вид таблицы

In [7]:
# Разделяем колонку с именем проекта на имя параметра и значение параметра
df[['Параметр', 'Значение']] = df['prj_name'].str.split('&', 1, expand=True)

# Удаляем исходную колонку
df = df.drop(columns=['prj_name'])

df_res = pd.pivot_table(df, values=['TgAffinPer'],
                        index=['Параметр', 'Значение'], 
                        columns=['tvNetName'])
df_res

TgAffinPer  \
tvNetName                                                                     СТС   
Параметр           Значение                                                         
4+                 Тотал                                               100.000000   
Возраст            18-34                                               124.635744   
                   35-44                                               152.186226   
                   4-17                                                 90.982592   
                   45-54                                               113.874465   
                   55+                                                  53.803269   
Затраты на питание Более трех четвертей                                110.728975   
                   Меньше четверти                                     104.308569   
                   Нет ответа                                           98.814079   
                   От половины до трех четвертей                        95.085723   
                   От четверти до половины                             100.027111   
Образование        Высшее                                               87.141639   
                   Начальное, среднее                                  104.250543   
Пол                Женщины                                              95.486707   
                   Мужчины                                             105.258133   
Род занятий        Домохозяйка, молодая мать                           169.944754   
                   Дошкольник, студент, учащийся                        87.458863   
                   Другое                                               67.386771   
                   Рабочий                                             133.257538   
                   Руководитель                                        119.741564   
                   Специалист, служащий                                109.136306   
Уровень дохода     Имеют возможность покупать дорогие вещи             100.376922   
                   Имеют возможность покупать только самое необход...  103.964416   
                   Хватает на еду и одежду, но не на дорогие вещи      100.499750   

                                                                                   
tvNetName                                                                     ТНТ  
Параметр           Значение                                                        
4+                 Тотал                                               100.000000  
Возраст            18-34                                               134.482774  
                   35-44                                               167.569789  
                   4-17                                                 48.747640  
                   45-54                                               125.951892  
                   55+                                                  57.015803  
Затраты на питание Более трех четвертей                                 73.146449  
                   Меньше четверти                                     108.652670  
                   Нет ответа                                           88.062403  
                   От половины до трех четвертей                       103.160689  
                   От четверти до половины                             100.335579  
Образование        Высшее                                              104.627088  
                   Начальное, среднее                                   98.470440  
Пол                Женщины                                              91.814766  
                   Мужчины                                             109.536064  
Род занятий        Домохозяйка, молодая мать                           153.487048  
                   Дошкольник, студент, учащийся                        50.805282  
                   Другое                                               76.885708  
           

## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [8]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('03_affinity'))
df_info = mtask.task_builder.get_report_info()
df_res.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()